# Introduction
A study of Adaptive max pooling

### Imports
Import libraries and write settings here.

In [6]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import torch.nn as nn
import torch.nn.functional as F

import torch
import torchvision
import torchvision.transforms as transforms
# import utils
from tensorboardX import SummaryWriter
import models
from train import get_transforms
from glob import glob
import time
import settings as s
import train as trn
import models

# Analysis/Modeling
Do work here

In [3]:
# using GPU?
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Choose a network type


In [7]:
## choose which network to use
# net = Net()
#path ="./model_weightsN.pth"

# net = models.Net_adaptive_pool_Small()
# path ="./model_weightsNAPS.pth"

net = models.Net_adaptive_pool()
path ="./model_weights_NAP.pth"

# load trained weights if there
try:
    net.load_state_dict(torch.load(path))
except FileNotFoundError:
    print("File " + s.PATH + " not present, you MUST retrain before you can test")
    
#Cuda or CPU?
net.to(device);

## Train the network

In [8]:
NUMB_EPOCHS = 5

In [9]:
# Define a Loss function and optimizer
# Let's use a Classification Cross-Entropy loss and SGD with momentum.
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#train and test
net.train()
net.to(device)
print("type of net is " + str(type(net)) + "\n")
trn.train_test(net, criterion, optimizer,device, numEpochs = NUMB_EPOCHS, path =path);

type of net is <class 'models.Net_adaptive_pool'>

Files already downloaded and verified
Files already downloaded and verified
Training loss: 0.003  accuracy: 0.713 Testing loss: 0.004  accuracy: 0.654
Training loss: 0.003  accuracy: 0.713 Testing loss: 0.004  accuracy: 0.656
Training loss: 0.003  accuracy: 0.712 Testing loss: 0.004  accuracy: 0.657
Training loss: 0.003  accuracy: 0.711 Testing loss: 0.004  accuracy: 0.656
Training loss: 0.003  accuracy: 0.716 Testing loss: 0.004  accuracy: 0.657
Finished Training


# Results
Show graphs and stats here

### Accuracy for each class in Test Set

In [10]:
net.eval()  # going to eval

print("type of net is " + str(type(net)) + "\n")
trn.check_each_class_accuracy(net, device)

Net_adaptive_pool(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
  (adaptive_mp2d): AdaptiveMaxPool2d(output_size=(5, 5))
  (fc1): Linear(in_features=250, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

type of net is <class 'models.Net_adaptive_pool'>

Files already downloaded and verified
Files already downloaded and verified
Accuracy of plane : 83 %
Accuracy of   car : 100 %
Accuracy of  bird : 50 %
Accuracy of   cat : 25 %
Accuracy of  deer : 50 %
Accuracy of   dog : 44 %
Accuracy of  frog : 60 %
Accuracy of horse : 80 %
Accuracy of  ship : 70 %
Accuracy of truck : 85 %


### Accuracy on unseen class instances

In [11]:
#set up network
device = "cpu"
net.to(device);# ; keeps this statement from priniting out the net config
net.eval();  # going to eval

filelist = glob(s.where_diff_sized_images_are + "*.jpg");

#### Fullsize Images

In [14]:
print("Evaluate original size");
from test_on_diff_sized_files import evaluate_dir;
evaluate_dir(filelist,net);

Evaluate original size
for image ./diff_sized/deer1.jpg the network predicted bird
for image ./diff_sized/deer2.jpg the network predicted bird
for image ./diff_sized/dog1.jpg the network predicted bird
for image ./diff_sized/dog2.jpg the network predicted bird
for image ./diff_sized/cat2.jpg the network predicted bird
for image ./diff_sized/car2.jpg the network predicted bird
for image ./diff_sized/cat1.jpg the network predicted bird
for image ./diff_sized/ship2.jpg the network predicted plane
for image ./diff_sized/plane2.jpg the network predicted plane
for image ./diff_sized/horse1.jpg the network predicted bird
for image ./diff_sized/bird2.jpg the network predicted bird
for image ./diff_sized/ship1.jpg the network predicted ship
for image ./diff_sized/horse2.jpg the network predicted dog

Inference took 1.4318373203277588 seconds
Network accuracy = 0.23076923076923078



#### 32x32 images (reduced from fullsize)
This is the size the network was trained on, this should be lower than above since we are losing information when downsampling the image

In [15]:
# Shrink images to 32,32 (orig cifar10 size) and evaluate
# accuracy should fall since we lost information when shrinking images
evaluate_dir(filelist, net, size = (32,32))

for image ./diff_sized/deer1.jpg the network predicted deer
for image ./diff_sized/deer2.jpg the network predicted deer
for image ./diff_sized/dog1.jpg the network predicted bird
for image ./diff_sized/dog2.jpg the network predicted bird
for image ./diff_sized/cat2.jpg the network predicted dog
for image ./diff_sized/car2.jpg the network predicted car
for image ./diff_sized/cat1.jpg the network predicted cat
for image ./diff_sized/ship2.jpg the network predicted ship
for image ./diff_sized/plane2.jpg the network predicted plane
for image ./diff_sized/horse1.jpg the network predicted deer
for image ./diff_sized/bird2.jpg the network predicted bird
for image ./diff_sized/ship1.jpg the network predicted ship
for image ./diff_sized/horse2.jpg the network predicted horse

Inference took 0.05644536018371582 seconds
Network accuracy = 0.6923076923076923



# Conclusions and Next Steps
Summarize findings here